In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import my_library.constants as constants

In [41]:
fit_systematics = pd.read_csv('/work/halld/home/viducic/systematic_errors/max_changes.csv')
signifigant_cuts = pd.read_csv('/work/halld/home/viducic/systematic_errors/signifigant_cuts.csv')
kstar_cuts = signifigant_cuts[signifigant_cuts['cut'].str.contains('kstar')]
signifigant_cuts = signifigant_cuts[signifigant_cuts['cut'].isin(['kinfit_cl', 'ppi', 'kp', 'ksp'])]
path_to_fit_result_files = '/work/halld/home/viducic/data/fit_params/{}/tf1_gaus_cross_section_values.csv'
df_pipkmks = pd.read_csv(path_to_fit_result_files.format('pipkmks'))
df_pipkmks['cross_section_percent_error'] = df_pipkmks['cross_section_error']/df_pipkmks['cross_section']
df_pimkpks = pd.read_csv(path_to_fit_result_files.format('pimkpks'))
df_pimkpks['cross_section_percent_error'] = df_pimkpks['cross_section_error']/df_pimkpks['cross_section']


In [42]:
cut_fit_errors = pd.DataFrame(columns=['channel', 'e', 't', 'stat_error', 'fit_error', 'cut_error', 'kstar_error', 'sys_error', 'total_error'])

In [ ]:
# TODO: loop over the cuts which has every e and t bin, dummy


In [43]:
grouped = signifigant_cuts.groupby(['e', 't', 'channel'])
for (e, t, channel), group in grouped:
    # print(e, t, channel)
    cut_errors = {}
    # print(group)
    cut_grouped = group.groupby('cut')
    for cut, cut_group in cut_grouped:
        cut_group['mag_change'] = np.where(True, abs(cut_group['percent_change']), 0)
        cut_errors[cut] = cut_group[cut_group['mag_change'] == cut_group['mag_change'].max()]['percent_change'].values[0]
    cut_error = np.sqrt(np.sum([cut_errors[cut]**2 for cut in cut_errors]))

    middle_of_bin = constants.T_CUT_DICT[t][0] + constants.T_WIDTH_DICT[t]/2
    # print(middle_of_bin)
    fit_error = fit_systematics[(fit_systematics['e'] == e) & (fit_systematics['t'] == t)]['max_percent_change'].values[0]
    if channel == 'pipkmks':
        stat_error = df_pipkmks[(df_pipkmks['beam_energy'] == e) & (df_pipkmks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
    else:
        stat_error = df_pimkpks[(df_pimkpks['beam_energy'] == e) & (df_pimkpks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
    sys_error = np.sqrt(fit_error**2 + cut_error**2)
    total_error = np.sqrt(stat_error**2 + sys_error**2)
    cut_fit_errors = cut_fit_errors.append({'channel': channel, 'e': e, 't': t, 'stat_error': stat_error, 'fit_error': fit_error, 'cut_error': cut_error, 'kstar_error': 0, 'sys_error': sys_error, 'total_error': total_error}, ignore_index=True)

    

In [44]:
# print(cut_fit_errors)
cut_fit_errors.to_csv('/work/halld/home/viducic/systematic_errors/cs_total_error.csv', index=False)